In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import glob
import time
import undetected_chromedriver as uc

In [2]:
# chrome_options = webdriver.ChromeOptions()
# prefs = {'download.default_directory' : os.getcwd()}
#chrome_options.add_argument("--headless")
# chrome_options.add_experimental_option('prefs', prefs)
# driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
# service = ChromeService(executable_path=ChromeDriverManager().install())
driver = uc.Chrome(headless=True,use_subprocess=True)


In [3]:
def openBudgetSite(driver, school_code, fiscal_year, budgetOrAllocation = 'budget'):
    if (budgetOrAllocation == 'allocation'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxyallocation/default.aspx")

    elif (budgetOrAllocation == 'budget'):
        driver.get("https://www.nycenet.edu/offices/d_chanc_oper/budget/dbor/galaxy/galaxybudgetsummaryto/default.aspx")
        
    else:
        raise ValueError('Budget or Allocation value must be either "budget" or "allocation"')
                         
    # get element 
    element = driver.find_element(By.XPATH,'//*[@id="School_Code"]')

    # send keys 
    element.send_keys(school_code)

    x = driver.find_element(By.XPATH,'//*[@id="Fiscal_Year"]')
    drop=Select(x)

    # select by visible text
    drop.select_by_visible_text(fiscal_year)

    driver.find_element(By.XPATH,'//*[@id="Enter"]').click()
    

    try:
        element = driver.find_element(By.XPATH,'//*[@id="message"]/div[1]/div[3]/h2/a')
        element.text.index(school_code)
        return True

    except NoSuchElementException:
        print('No ' + budgetOrAllocation + ' data could be found for school code ' + school_code + ' in the year ' + year)
        return False
        
    except ValueError:
        print('Given School code and school code for retrieved data do not match')
        return False


def allocationPageScraper(driver, school_code, fiscal_year):
    list_of_dfs = pd.read_html(driver.page_source)
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'allocation_category', 'amount'])

    for df in list_of_dfs:
        df = df.rename(columns={df.columns[0]: 'allocation_category', df.columns[1]: 'amount'})
        df = df[df["allocation_category"].str.contains("total", case = False)==False]
        df['location_code'] = school_code
        df['fiscal_year'] = fiscal_year
        final_df = pd.concat([final_df, df])
    
    return final_df
    
    
def budgetPageScraper(driver, school_code, fiscal_year):
    section_titles = driver.find_elements(By.CLASS_NAME , 'TO_Section')
    list_of_dfs = pd.read_html(driver.page_source)
    
    final_df = pd.DataFrame(columns=['location_code', 'fiscal_year', 'budget_category', 'budget_assignment', 'num_positions', 'service_type', 'amount'])

    for i,df in enumerate(list_of_dfs):
        if (len(df.columns) == 2):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['num_positions'] = None
            df['service_type'] = None
            final_df = pd.concat([final_df, df])
        
        elif (len(df.columns) == 3):
            if (section_titles[i].text == 'School Site'):
                continue
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'num_positions', df.columns[2]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            df['service_type'] = None
            final_df = pd.concat([final_df, df])

        elif (len(df.columns) == 4):
            df = df.rename(columns={df.columns[0]: 'budget_assignment', df.columns[1]: 'service_type', df.columns[2]: 'num_positions', df.columns[3]: 'amount'})
            df["budget_assignment"] = df["budget_assignment"].astype(str)
            df = df[df["budget_assignment"].str.contains("total", case = False)==False]
            df = df[df["service_type"].str.contains("total", case = False)==False]
            df['location_code'] = school_code
            df['fiscal_year'] = fiscal_year
            df['budget_category'] = section_titles[i].text
            final_df = pd.concat([final_df, df])
            
    return final_df

In [4]:
# read in school codes
school_data = pd.read_csv("../school_code_budget_extraction.csv")

In [5]:
school_data

,school_code,2018,2019,2020,2021,2022,2023,2024
0,M040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M142,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,K953,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,K284,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,K898,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1612,X469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1613,X721,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1614,X723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1615,X754,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import time

budget_data = pd.DataFrame()
allocation_data = pd.DataFrame()
#budget_data = pd.read_csv('../data/budget_all_2024.csv')
#allocation_data =  pd.read_csv('../data/allocation_all_2024.csv')

years = ['2018', '2019', '2020', '2021', '2022', '2023', '2024']
c=0
for index, row in school_data.iterrows():
    if (not pd.isnull(row["school_code"])) and row["2024"] != "done" : #some of the rows in school data are unfortunately empty, so just skip them. AND if the row is not Completely "done" yet then do it whole again
        c+=1
        if ((c%50)==0):
            budget_data.to_csv('../data/budget_working.csv', index=False)
            allocation_data.to_csv('../data/allocation_working.csv', index=False)
        if c==300:
            time.sleep(300) # sleep for 5 mins 
            driver = uc.Chrome(headless=True,use_subprocess=True)
            c=0
        print(row)
        school = row["school_code"]
        for year in years:
            if (openBudgetSite(driver, school, year, 'budget')): # ToDo: split budget from allocation and use 2 different drivers to reduce rate limit and parallelize
                print(year +  ' Budget Data Opened')
                budget_data = pd.concat([budgetPageScraper(driver, school, year), budget_data])
            if (openBudgetSite(driver, school, year, 'allocation')):
                print(year +  ' Allocation Data Opened')
                allocation_data = pd.concat([allocationPageScraper(driver, school, year), allocation_data])
            school_data[year][index]="done"


In [103]:
budget_data.to_csv('../data/budget_all_2024.csv',index=False)

In [104]:
allocation_data.to_csv("../data/allocation_all_2024.csv",index=False)

In [105]:
school_data.to_csv("../school_code_budget_extraction.csv",index=False)

In [ ]:
### Note: for missing schools i.e. schools with no data in budget_all.csv and allocation_call_csv its possible something went wrong.
# for this, the school_code_budget_extraction should show all dates marked "done" but nothing in the main sheets.